In [1]:
import sys
import os
sys.path.append(os.path.abspath("python-iso18245-master"))
import iso18245
import pandas as pd
from typing import List, Dict, Any

In [ ]:
print(dir(iso18245))  # This will show us all available attributes/methods

In [5]:
mccs = pd.DataFrame(iso18245.get_all_mccs_dict()).drop(columns=['visa_description'])
mcc_visa = pd.read_csv("mcc_visa_pdf_data.csv", dtype={"MCC":str})
mcc_visa.columns = ['visa_' + str.lower(col) for col in mcc_visa.columns]

In [6]:
mccs_merged =mccs.merge(mcc_visa[['visa_mcc','visa_mcc_title','visa_mcc_description','visa_included_in_mcc']], left_on="mcc", right_on="visa_mcc", how = 'outer')

In [ ]:
dict(mccs_merged.iloc[700:710].iloc[0])